<a href="https://colab.research.google.com/github/shaguftah10sep/assignments/blob/main/Bank_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install pyspark if not already installed
!pip install pyspark

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum as spark_sum, trim
from pyspark.sql.window import Window
from google.colab import files

# Initialize Spark session
spark = SparkSession.builder.appName("TransactionAnalysis").getOrCreate()

# Upload the file
uploaded = files.upload()

# Get the filename from the uploaded files
file_path = list(uploaded.keys())[0]

# Load the Excel file into a Pandas DataFrame
df_pandas = pd.read_excel(file_path, sheet_name='AccountTransaction')

# Convert the TransactionDate to proper datetime format
df_pandas['TransactionDate'] = pd.to_datetime(df_pandas['TransactionDate'], format='%Y%m%d')

# Convert Pandas DataFrame to PySpark DataFrame
df_spark = spark.createDataFrame(df_pandas)

# Strip spaces from TransactionType and convert TransactionDate to DateType for easier sorting
df_spark = df_spark.withColumn("TransactionType", trim(col("TransactionType")))
df_spark = df_spark.withColumn("TransactionDate", col("TransactionDate").cast("date"))

# Define a window partitioned by AccountNumber and ordered by TransactionDate
windowSpec = Window.partitionBy("AccountNumber").orderBy("TransactionDate")

# Add a column to calculate the adjusted amount (positive for Credit, negative for Debit)
df_spark = df_spark.withColumn("AdjustedAmount", when(col("TransactionType") == "Credit", col("Amount"))
                                             .otherwise(-col("Amount")))

# Calculate running total (current balance) using the window function
df_spark = df_spark.withColumn("CurrentBalance", spark_sum("AdjustedAmount").over(windowSpec))

# Select and display the final output
df_spark_final = df_spark.select("TransactionDate", "AccountNumber", "TransactionType", "Amount", "CurrentBalance")
df_spark_final.show()



Saving data.xlsm to data (3).xlsm
+---------------+-------------+---------------+------+--------------+
|TransactionDate|AccountNumber|TransactionType|Amount|CurrentBalance|
+---------------+-------------+---------------+------+--------------+
|     2023-01-01|          100|         Credit|  1000|          1000|
|     2023-01-02|          100|         Credit|  1500|          2500|
|     2023-01-03|          100|          Debit|  1000|          1500|
|     2023-01-02|          200|         Credit|  3500|          3500|
|     2023-01-03|          200|          Debit|  2000|          1500|
|     2023-01-04|          200|         Credit|  3500|          5000|
|     2023-01-13|          300|         Credit|  4000|          4000|
|     2023-01-14|          300|          Debit|  4500|          -500|
|     2023-01-15|          300|         Credit|  1500|          1000|
+---------------+-------------+---------------+------+--------------+

